In [1]:
#!pip install datasets transformers rouge-score #nltk
#!pip install transformers[torch]
#!pip install --upgrade Torch 
##!pip install tensorboardX
#!pip install tensorboard

In [2]:
from datasets import load_dataset, load_metric
import pandas as pd
from sklearn.utils import shuffle
import torch
import gc

In [3]:
def cleanup():
    gc.collect()
    torch.cuda.empty_cache()
    
cleanup()

In [4]:
import torch
torch.cuda.empty_cache()
cleanup()

In [5]:
#parallel corpus
dataset_en = load_dataset('s-nlp/paradetox')
#dataset_ru = load_dataset('s-nlp/ru_paradetox')

#df = pd.read_csv('../../data/input/train.tsv', sep='\t')
#df = df.fillna('')
dataset_en['train']['en_toxic_comment'][-5:], dataset_en['train']['en_neutral_comment'][-5:]

(['when they do shit like this .',
  'but if saying " fuck that group " is much more offensive for you , go ahead .',
  'it hurts how judgemental assholes view them , and no one gives a shit about that .',
  'shit we probably literally blow that up in a week .',
  'if anyone deserved it , it was this shit bag .'],
 ['when they do stuff like this',
  'but if saying" that group is bad" is much more offensive for you, go ahead.',
  'It hurts how judgemental that people view them and no one cares about that',
  'We probably litteralky blow that up in a week.',
  'if anyone deserved it , it was this bad bag .'])

In [6]:
#Preparing data
train_toxic_en = dataset_en['train']['en_toxic_comment']
train_neutral_en = dataset_en['train']['en_neutral_comment']
val_toxic_en = dataset_en['train']['en_toxic_comment']
val_neutral_en = dataset_en['train']['en_neutral_comment']

#Dataset train/validation/test split
df_train_en = pd.DataFrame({
    'toxic_comment': train_toxic_en,
    'neutral_comment': train_neutral_en
})
df_train_en = shuffle(df_train_en)

df_val_en = pd.DataFrame({
    'toxic_comment': val_toxic_en,
    'neutral_comment': val_neutral_en
})
df_val_en = shuffle(df_val_en)
df_val_en

,toxic_comment,neutral_comment
5432,bish u luucky grr i will find a way fuck never...,u luucky grr i will find a way never say never
5262,damn straight ! ! dd when you said it to me on...,damn straight ! ! dd when you said it to me on...
15731,i aaaaaaint havin that shit !,I don't want it.
7649,rt all fun and games are over when this bitch ...,All the fun and games are over when she get home.
8648,everybody getting on my fuking nerves,Everybody is getting on my nerves.
...,...,...
13681,anyone who doesn 't kiss the ayatollah 's ass ...,anyone who doesn 't kiss the ayatollah must be...
1836,can someone explain to these idiot jurors that...,Can someone explain to these jurors that the d...
7907,very welcomee keep up the goodwork fuck the ha...,very welcomee keep up the goodwork ! < 79
11372,looks pretty fucking accurate to me .,Looks so accurate to me


In [7]:
df_train_en['toxic_comment'][0], df_val_en['toxic_comment'][0]

('he had steel balls too !', 'he had steel balls too !')

In [8]:
#import nltk
#nltk.download('punkt')
#import string
from transformers import AutoTokenizer

In [9]:
#model_checkpoint = "t5-base"

#English
model_checkpoint = 'facebook/bart-base'
model_name = 'SkolkovoInstitute/bart-base-detox'


tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [10]:
max_input_length = 512
max_target_length = 512

train_en_tokenized = tokenizer(df_train_en['toxic_comment'].tolist(), max_length=max_input_length, truncation=True)
val_en_tokenized = tokenizer(df_val_en['toxic_comment'].tolist(), max_length=max_input_length, truncation=True)

# Setup the tokenizer for targets
with tokenizer.as_target_tokenizer():
    train_labels = tokenizer(df_train_en["neutral_comment"].tolist(), max_length=max_target_length, truncation=True)
    val_labels = tokenizer(df_val_en["neutral_comment"].tolist(), max_length=max_target_length, truncation=True)
    
train_en_tokenized['labels'] = train_labels['input_ids']
val_en_tokenized['labels'] = val_labels['input_ids']

#X_train_ru_tokenized["labels"] = y_train_ru_tokenized["input_ids"]
#X_val_ru_tokenized["labels"] = y_train_ru_tokenized["input_ids"]


/home/khantr/detox/lib64/python3.9/site-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [11]:
from datasets import Dataset
ds_train_en_tokenized = Dataset.from_dict(train_en_tokenized)
ds_val_en_tokenized = Dataset.from_dict(val_en_tokenized)

Finetuen Base-Bert Detox

In [12]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [13]:
#model_name = "t5-base-medium-title-generation"
model_dir = 'models/' + model_name
!rm -r {model_dir}

rm: cannot remove 'models/SkolkovoInstitute/bart-base-detox': No such file or directory


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [14]:
batch_size = 1 #8

args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="steps",
    eval_steps=100,
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="steps",
    save_steps=200,
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    report_to="tensorboard"
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [16]:
import numpy as np

metric = load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
 
    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

/tmp/ipykernel_38661/694220924.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")
/home/khantr/detox/lib64/python3.9/site-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [17]:
cleanup()

In [18]:
# Function that returns an untrained model to be trained
def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=ds_train_en_tokenized,
    eval_dataset=ds_val_en_tokenized,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [19]:
#torch.cuda.memory_summary(device=None, abbreviated=False)

In [20]:
# Start TensorBoard before training to monitor it in progress
%load_ext tensorboard
%tensorboard --logdir "models/run/"

Reusing TensorBoard on port 6006 (pid 1766381), started 23:17:14 ago. (Use '!kill 1766381' to kill it.)

In [21]:
torch.cuda.empty_cache()

In [22]:
trainer.train()

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
100,1.991700,1.386320,77.292200,64.808500,76.819700,76.815100,14.131900
200,1.565300,1.637515,77.712000,65.355400,77.256100,77.256300,14.970000
300,1.652700,1.290758,79.182200,67.125700,78.695000,78.699100,14.001600
400,1.602200,1.386856,79.048800,67.249100,78.579500,78.581700,14.666900
500,1.327300,1.374563,79.664800,67.901100,79.187500,79.197000,14.496900
600,1.420600,1.314055,79.208400,67.659600,78.771000,78.770900,13.791300
700,1.546400,1.252452,79.200100,67.246700,78.731600,78.728100,13.671400
800,1.643700,1.284805,80.085000,68.369000,79.609100,79.607600,14.147400
900,1.678500,1.246119,79.490700,67.787300,79.021700,79.026700,13.571600
1000,1.276800,1.294329,79.594000,68.019600,79.137200,79.140100,13.991200


/home/khantr/detox/lib64/python3.9/site-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
/home/khantr/detox/lib64/python3.9/site-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the gen

KeyboardInterrupt: 

In [23]:
trainer.save_model()

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


In [24]:
#model_name = "t5-base-medium-title-generation/checkpoint-8800"
model_path = "models/"+model_name
print(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

max_input_length = 512

models/SkolkovoInstitute/bart-base-detox


In [25]:
inputs = ['when they do shit like this .']

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]

print(decoded_output)

When they do things like this. I don't care.


In [48]:
test = pd.read_csv('data/test_without_answers.tsv', encoding='utf-8', sep='\t')
test = test[test['lang']=='en']
test_en = list(test['toxic_sentence'])
test_en[:2], len(test_en)

(['mandated  and " right fucking now " would be good .',
  "&gt i wouldn 't care how ignorant you are in you weren 't pretending to know shit ."],
 600)

In [53]:
result = []
for item in test_en:
    inputs = tokenizer(item, max_length=max_input_length, truncation=True, return_tensors="pt", padding=True)
    output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
    decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    result.append(decoded_output)

In [56]:
lang = ['en'] * len(test_en)

In [61]:
output = pd.DataFrame(
    {'toxic_sentence': test_en, 'neutral_sentence': result, 'lang': lang}
)

In [66]:
output.to_csv('data/test_witanswers_en.tsv', encoding='utf-8', sep='\t')